导入库，读取数据

In [ ]:
import lightning.pytorch as pl
import matplotlib.pyplot as plt
import pandas as pd
from lightning.pytorch.callbacks import ModelCheckpoint
from dataset import ElectricityDataModule
from tpa_lstm import TPALSTM
from GA_BP import GABP
#from tpa_lstm import TPALSTM
import torch
from torch import nn
from lightning.pytorch.callbacks import EarlyStopping
#data_df = pd.read_csv('ture_cd.csv')
#data_df = pd.read_csv('testdata.csv')
pred_horizons=[12,24,48,96]
for pred_horizon in pred_horizons:
    for i in range(15):
        data_df = pd.read_excel('cd-data.xlsx',sheet_name=str(i+1))
        data_df.drop(columns=['date'],inplace=True)
        num_features = data_df.shape[1]
        data_splits = {
                "train": 0.7,
                "val": 0.15,
                "predict": 0.15
            }
        #pred_horizon = 12
        batch_size=32
        hid_size = 64
        n_layers = 3
        num_filters = 3

        time_windows=[60]
        run_name = f"{pred_horizon}-TPA-化工-Y"+str(i)
        num_y=1

        for time_window in time_windows:

            elec_dm = ElectricityDataModule(
                dataset_splits=data_splits,
                batch_size=batch_size,
                window_size=time_window,
                pred_horizon=pred_horizon,
                data_style="custom",
                data=data_df
            )

            name = f'{run_name}-TPA-LSTM-'+str(time_window)

            

            checkpoint_loss_tpalstm = ModelCheckpoint(
                dirpath=f"NewCheckpoints/{run_name}",
                filename=name,
                save_top_k=1,
                monitor="val/loss",
                mode="min"
            )


            early_stop_callback = EarlyStopping(
                monitor='val/loss',  # 指定监控指标
                min_delta=0.00,  # 指标提升的最小变化量
                patience=2,  # 没有提升时的等待轮数
                verbose=True,
                mode='min'
            )

            tpalstm_trainer = pl.Trainer(
                max_epochs=100,
                # accelerator='gpu',
                callbacks=[checkpoint_loss_tpalstm,early_stop_callback],
                #callbacks=[checkpoint_loss_tpalstm],
                strategy='auto',
                devices=1,
                # logger=wandb_logger_tpalstm
            )

            # tpa_lstm = TPALSTM(
            #     input_size=num_features,
            #     hidden_size=hid_size,
            #     output_horizon=pred_horizon,
            #     num_filters=num_filters,
            #     obs_len=time_window,
            #     n_layers=n_layers,
            #     lr=1e-3,
            #     num_y=num_y
            #     #num_heads=2
            # )
            tpa_lstm = GABP(
                input_size=num_features,
                hidden_size=hid_size,
                n_layers=n_layers,
                output_horizon=pred_horizon,
                num_y=num_y,
                num_filters=num_filters,
                obs_len=time_window
            )
            # tpa_lstm.eval()
            # input=torch.rand(batch_size,time_window,num_features)
            # out=tpa_lstm(input)
            
            tpalstm_trainer.fit(tpa_lstm, elec_dm)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
d:\Anaconda3\envs\env_ML2\lib\site-packages\lightning\pytorch\trainer\connectors\logger_connector\logger_connector.py:75: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default

  | Name      | Type                     | Params
-------------------------------------------------------
0 | hidden    | Linear                   | 1.2 K 
1 | relu      | ReLU                     | 0     
2 | lstm      | LSTM                     | 96.0 K
3 | attention | TemporalPatternAttention | 4.7 K 
4 | mlp_out   | Se

Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

d:\Anaconda3\envs\env_ML2\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.


d:\Anaconda3\envs\env_ML2\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.


Epoch 0: 100%|██████████| 263/263 [01:02<00:00,  4.18it/s, v_num=1098, val/loss=6.000, val/corr=0.00876, val/rse=967.0, train/loss=0.535, train/corr=0.00723, train/rse=59.40]

...

Metric val/loss improved. New best score: 0.872


Epoch 1:  18%|█▊        | 47/263 [00:10<00:50,  4.28it/s, v_num=1123, val/loss=0.872, val/corr=0.00983, val/rse=226.0, train/loss=0.916, train/corr=-0.00348, train/rse=78.90] 

d:\Anaconda3\envs\env_ML2\lib\site-packages\lightning\pytorch\trainer\call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name      | Type                     | Params
-------------------------------------------------------
0 | hidden    | Linear                   | 1.2 K 
1 | relu      | ReLU                     | 0     
2 | lstm      | LSTM                     | 95.5 K
3 | attention | TemporalPatternAttention | 4.7 K 
4 | mlp_out   | Sequential               | 2.9 K 
5 | linear    | Linear                   | 1.6 K 
6 | criterion | MSELoss                  | 0     
-------------------------------------------------------
105 K     Trainable params
0         Non-trainable params
105 K     Total params
0.423     Total estimated model params size (MB)


d:\Anaconda3\envs\env_ML2\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.
d:\Anaconda3\envs\env_ML2\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.


Epoch 0:  13%|█▎        | 34/263 [00:08<00:55,  4.15it/s, v_num=1124]

预测

In [ ]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"  # 解决MacOS系统下的问题
import lightning.pytorch as pl
import matplotlib.pyplot as plt
import pandas as pd
from lightning.pytorch.callbacks import ModelCheckpoint
from dataset import ElectricityDataModule
from tpa_lstm import TPALSTM
#from tpa_lstm import TPALSTM
import torch
from torch import nn
from lightning.pytorch.callbacks import EarlyStopping

def MAE(ypred, ytrue):
    absolute_errors = torch.abs(ypred - ytrue)
    # 求和得到总的MAE
    mae = torch.sum(absolute_errors)
    # 如果有多个batch或多个样本，需要进行均分
    temp=1
    for i in range(len(ypred.size())):
        temp=temp*ypred.size(i)
    mae = mae / temp  # 假设第一维是batch size
    return mae
                      
tpalstm_trainer = pl.Trainer(
    max_epochs=100,
    # accelerator='gpu',
    #callbacks=[checkpoint_loss_tpalstm],
    strategy='auto',
    devices=1,
    # logger=wandb_logger_tpalstm
)

MSE = nn.MSELoss()


  


#data_df = pd.read_csv('ture_cd.csv')
#data_df = pd.read_csv('testdata.csv')

mse1=[]
mae1=[]
pred_horizons=[12,24,48,96]
num_y=1
for pred_horizon in pred_horizons:
    Yture1=[]
    Ypred1=[]
    for ii in range(15):
        data_df = pd.read_excel('cd-data.xlsx',sheet_name=str(ii+1))
        data_df.drop(columns=['date'],inplace=True)
        num_features = data_df.shape[1]
        data_splits = {
                "train": 0.7,
                "val": 0.15,
                "predict": 0.15
            }
        batch_size=32
        hid_size = 64
        n_layers = 3
        num_filters = 3

        time_window=10
        time_windows=[96]
        run_name = f"{pred_horizon}-TPA-化工-Y"+str(ii)    
        folder_path='CD-'+str(pred_horizon)

        for time_window in time_windows:

            

            elec_dm = ElectricityDataModule(
                dataset_splits=data_splits,
                batch_size=batch_size,
                window_size=time_window,
                pred_horizon=pred_horizon,
                data_style="custom",
                data=data_df,
                num_y=num_y
            )
            
            elec_dm.setup("predict")

            pred_dl = elec_dm.predict_dataloader()

            name = f'{run_name}-TPA-LSTM-'+str(time_window)
            run_to_load = run_name
            model_path = f"NewCheckpoints/{run_to_load}/{name}.ckpt"
            tpa_lstm = TPALSTM.load_from_checkpoint(model_path)
            y_pred = tpalstm_trainer.predict(tpa_lstm, pred_dl)

            batch_idx = 0

            Yture=[]
            Ypred=[]


            batch_idx = 0
            for i, batch in enumerate(pred_dl):
                inputs, labels = batch
                X, ytrue = inputs[batch_idx][:, -1],  labels.squeeze()
                ypred = y_pred[i].squeeze()
                Yture.append(ytrue)
                Ypred.append(ypred)
                #X = X.cpu().numpy()
            if len(Yture[-1].size())!=len(Yture[-2].size()):
                Yture[-1]=Yture[-1].unsqueeze(0)
                Ypred[-1]=Ypred[-1].unsqueeze(0)
            Yture=torch.cat(Yture,dim=0)
            Ypred=torch.cat(Ypred,dim=0)
           
        Yture1.append(Yture.unsqueeze(2))
        Ypred1.append(Ypred.unsqueeze(2))
    Yture1=torch.cat(Yture1,dim=2)
    Ypred1=torch.cat(Ypred1,dim=2)
    if not os.path.exists(folder_path):
            os.makedirs(folder_path)
    torch.save(Yture1,folder_path+'/true.npy')
    torch.save(Ypred1,folder_path+'/pred.npy')
    mse2=MSE(Ypred1,Yture1)
    mae2=MAE(Ypred1,Yture1)
    print(folder_path,mse2)
    print(folder_path,mae2)
    a=1
